In [5]:
#!/usr/bin/env python
# coding: utf-8

In[ ]:

!pip install -r requirements.txt<br>
!pip install pipreqs<br>
print(os.listdir('data_tg'))<br>
!pipreqs --force<br>
!jupyter nbconvert --to script 'Train.ipynb'<br>
!pip install netCDF4<br>
!pip install h5py<br>
!pip install pyhdf<br>
!pip install basemap<br>
!pip install pydap<br>
!pip install xarray<br>
!pip install pygrib<br>
!pip install opencv-python<br>
!pip install zstandard

In[ ]:

In [18]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GroupKFold
import lightgbm as lgb
import warnings
import sklearn
from IPython.display import display
import cv2
import xarray as xr
import pydap
import h5py
from pyhdf.SD import SD, SDC
from mpl_toolkits.basemap import Basemap
import matplotlib as mpl
from joblib import Parallel, delayed
import random
import secrets
import datetime
import pygrib
import time
import zstandard as zstd
import tarfile
import io
import requests
import pickle
import subprocess
import json
import os
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import numpy as np
get_ipython().system('pip install opencv-python-headless')

  Using cached opencv_python_headless-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.9.0.80-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (49.6 MB)


In[ ]:

In[ ]:

In[ ]:

In [19]:
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.max_open_warning'] = 100

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
pd.set_option('display.max_columns', 100)

In[ ]:

In[ ]:

s3 = boto3.client('s3')<br>
sqs = boto3.client('sqs')

In[ ]:

In [ ]:
zd = zstd.ZstdDecompressor()

In[ ]:

In[ ]:

In[ ]:

In [ ]:
SEED = datetime.datetime.now().microsecond

In[ ]:

In [ ]:
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)
run_label = secrets.token_hex(3)

In[ ]:

In[ ]:

In[ ]:

In [ ]:
dataset = 'tg'

In[ ]:

In [ ]:
ASSIM = True

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
labels = pd.read_csv('data_{}/train_labels.csv'.format(dataset))
grid = pd.concat((
    pd.read_csv('data_tg/grid_metadata.csv'),
    # pd.read_csv('data_pm/grid_metadata.csv')
)).drop_duplicates().reset_index(drop=True)

In[ ]:

In [ ]:
submission = pd.read_csv('data_{}/submission_format.csv'.format(dataset))
files = pd.read_csv('data_{}/{}_satellite_metadata{}.csv'.format(
                    dataset, *(('pm25', '') if dataset == 'pm'
                               else ('no2', '_0AF3h09'))))

In[ ]:

In [ ]:
labels['location'] = grid.set_index(
    'grid_id')['location'].reindex(labels.grid_id).values
labels['datetime'] = pd.to_datetime(labels.datetime)

In[ ]:

In [ ]:
submission['location'] = grid.set_index(
    'grid_id').location.reindex(submission.grid_id).values

In[ ]:

loc_dict = {'la': 'Los Angeles (SoCAB)', 'tpe': 'Taipei', 'dl': 'Delhi'}

In [ ]:
loc_dict = {'la': 'Los Angeles (SoCAB)'}

In[ ]:

In [ ]:
coords = defaultdict(list)
for city, gcity in loc_dict.items():  # ['Delhi', 'Taipei', 'LA']:
    # gcity = [c for c in grid.location.unique() if 'Los A' in c][0] if city not in grid.location.unique() else city
    grid_points = grid[grid.location == gcity]
    for e in grid_points.itertuples():
        coords[city].append((e.grid_id, *np.array([(float(p.split(' ')[0]),
                                                    float(p.split(' ')[1]))
                                                   for p in e.wkt[10:].split(', ')[:4]]).mean(axis=0).round(4).tolist()))

In[ ]:

In[ ]:

In [ ]:
def cleanDict(d):
    return {
        k: cleanDict(v) for k,
        v in d.items()} if isinstance(
        d,
        defaultdict) else d

In[ ]:

In [ ]:
assert sum([len(v) for c, v in coords.items()]) == grid.grid_id.nunique()
grid.grid_id.nunique()

In[ ]:

In [ ]:
coords = cleanDict(coords)

In[ ]:

In[ ]:

In[ ]:

t = datetime.datetime(2016, 1, 1)<br>
# t = datetime.datetime(2020, 5, 26)<br>
while t < datetime.datetime.now() - datetime.timedelta(days = 1):<br>
    print(t)<br>
    for tag in ifs_tags:<br>
    # for tag in ifs_forecasts:# ifs_tags:#[:1]:<br>
        domain = 'ec.oper.an.sfc'<br>
        # domain = 'ec.oper.fc.sfc'<br>
        file =  '{}/{}/{}.{}.regn1280sc.{}.nc'.format(domain,<br>
                        datetime.datetime.strftime(t, '%Y%m'),<br>
                        domain, tag,<br>
                        datetime.datetime.strftime(t, '%Y%m%d') )<br>
        if file.split('/')[-1] not in existing_files:<br>
            sqs.send_message(<br>
                QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-ifs',<br>
                MessageBody=json.dumps({'file': file}<br>
            ) )<br>
        # else:<br>
        #     print(file, 'exists')<br>
    t += datetime.timedelta(days = 1)<br>
    # break;

In[ ]:

In[ ]:

In[ ]:

In [ ]:
fields = ['nitrogendioxide_tropospheric_column',
          'nitrogendioxide_tropospheric_column_precision',
          'air_mass_factor_troposphere',
          'air_mass_factor_total']

In[ ]:

In[ ]:

In [ ]:
trop_scr_dir = 'tropomi_data/secure.txt'
trop_secure = dict([e.split('=')
                   for e in open(trop_scr_dir, 'r').read().split('\n')])

In[ ]:

In [ ]:
download_dir = '/tmp/'

In [ ]:
def loadFile(row):
    for i in range(1):
        try:
            values = {
                'email': trop_secure['username'],
                'passwd': trop_secure['password'],
                'action': 'login'}
            login_url = 'https://urs.earthdata.nasa.gov'
            ret = requests.post(login_url, data=values)
            if ret.status_code == 200:
                print("Login successful.")
            else:
                print("Bad Authentication")
                return None
        except Exception as e:
            print(e)
            time.sleep(i)
    zc = zstd.ZstdCompressor(level=15)
    os.makedirs(download_dir, exist_ok=True)
    filename, url, sz = [row[k]
                         for k in ['granule_id', 'us_url', 'granuleSize']]
    # print(filename, url, sz)
    file = '/tmp/' + filename
    try:
        print("Downloading", filename)
        if os.path.exists(file):
            print(f"{file} already exists. Skipping download.")
            pass
        with requests.get(url, cookies=ret.cookies,
                          allow_redirects=True, stream=True) as r:
            r.raise_for_status()
            with open(file, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024 * 1024):
                    f.write(chunk)
        print(f"Downloaded and compressed {filename} to {file}")
    except requests.RequestException as e:
        print(f"Error downloading {url}: {e}")
    return file

In[ ]:

In[ ]:

In[ ]:

In [ ]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings(
    'ignore',
    category=sklearn.exceptions.ConvergenceWarning)
warnings.filterwarnings('ignore', category=sklearn.exceptions.FitFailedWarning)

In[ ]:

In[ ]:

In[ ]:

In [ ]:
fields = ['nitrogendioxide_tropospheric_column',
          'nitrogendioxide_tropospheric_column_precision',
          'air_mass_factor_troposphere',
          'air_mass_factor_total']

In[ ]:

In[ ]:

In [ ]:
def run(hdf, location, fine=True):
    zones = {}  # defaultdict(dict)
    sat_data = defaultdict(lambda: defaultdict(dict))
    hp = hdf['PRODUCT']
    lat = hp['latitude'][:][0]  # .values
    lon = hp['longitude'][:][0]  # .values
    for field in fields:
        v = hp[field][:][0]
        data = np.ma.masked_array(
            v, (v == v.max()) | (
                v == v.min())).clip(
            0, None)
        assert data.shape == lat.shape
        for grid_id, plon, plat in coords[location]:
            for r in ([0.07, 0.1, 0.14, 0.2, 0.3, 0.5, 1, 2]
                      if fine else [0.1, 0.25, 0.5, 1, 2, ]):
                if (grid_id, r) not in zones:
                    zones[(grid_id, r)] = (lat - plat) ** 2 + \
                        (lon - plon) ** 2 < r ** 2
                zone = zones[(grid_id, r)]
                ct = data[zone].count()
                m = data[zone].mean() if ct > (0 if 'fine' else 3) else np.nan
                s = data[zone].std() if ct >= 3 else np.nan
                sat_data[grid_id][field + '_mean{}'.format(r)] = m
                sat_data[grid_id][field + '_stdev{}'.format(r)] = s
                sat_data[grid_id][field + '_count{}'.format(r)] = ct
                if '2' in grid_id:  # .startswith('9'):
                    print(field, '_count{}'.format(r), ct, m, s)
    return sat_data

In[ ]:

In [ ]:
def runTropomi(row, fine=True):
    assert row['product'].startswith('tropomi')

    # Presumably, loadFile is a function you've defined elsewhere to load the
    # data file
    file = loadFile(row)
    hdf = h5py.File(file, 'r')

    # Assuming run is a function you've defined that processes the hdf file
    # based on the location and 'fine' parameter
    sat_data = run(hdf, row['location'], fine)
    output = row.copy()

    # Assuming cleanDict is a function you've defined to clean or process the
    # satellite data in some way
    output['d1'] = cleanDict(sat_data)

    # Compress the serialized output
    zc = zstd.ZstdCompressor(level=15)
    o = pickle.dumps(output)
    c = zc.compress(o)

    # Save the compressed data locally
    local_dir = "path/to/your/local/directory"  # Specify your directory here
    filename = 'aqi_tropomi{}-{}.zst'.format(
        '-fine' if fine else '', os.path.basename(file))
    local_path = os.path.join(local_dir, filename)
    with open(local_path, 'wb') as f:
        f.write(c)
    os.remove(file)
    return {
        'statusCode': 200,
        'len': len(o),
        'clen': len(c),
        'body': json.dumps('Success'),
        'localPath': local_path  # Optionally return the path where the file was saved
    }

In[ ]:

In[ ]:

In[ ]:

runTropomi(rows[-4], fine = True)

In[ ]:

In[ ]:

rows[-1]<br>
In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    '# Parallel(os.cpu_count() #// 2\n#             )(delayed(runTropomi)(row) for row in rows)\n')

In[ ]:

In[ ]:

In[ ]:

d = pd.to_datetime(files[(files['product'] == 'misr')<br>
       & (files.location == 'la')].time_end)<br>
d = d.sort_values().reset_index(drop = True)<br>
(d.diff().dt.total_seconds() / (60*60*24)).iloc[:].plot()

In[ ]:

In[ ]:

In[ ]:

coords['LA']

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

sqs.send_message(<br>
    QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-gfs25-extra',<br>
    MessageBody=json.dumps((2020, 1, 1, 6, 0))<br>
)

t = datetime.datetime(2016, 10, 1)<br>
while t < datetime.datetime.now():<br>
    print(t)<br>
    for hr in range(0, 24, 6):<br>
        for fwd in (0, ):<br>
            sqs.send_message(<br>
                QueueUrl='https://sqs.us-east-2.amazonaws.com/815054066888/aqi-gfs25-extra',<br>
                MessageBody=json.dumps((t.year, t.month, t.day, hr, fwd))<br>
            )<br>
    t += datetime.timedelta(days = 1)

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

[e[0] for e in pickle.loads(zd.decompress(<br>
    s3.get_object(Bucket = 'projects-v',<br>
              Key = 'aqi/gfs-5/{}'.format('gfs.0p25.2016100218.f000.grib2'))['Body'].read()<br>
)) if 'LA' in e[1]]

[e[0] for e in pickle.loads(<br>
    s3.get_object(Bucket = 'projects-v',<br>
              Key = 'aqi/gfs/{}'.format('gfs.0p25.2016100112.f000.grib2'))['Body'].read()<br>
) if 'LA' in e[1]]

[e for e in pickle.loads(<br>
    s3.get_object(Bucket = 'projects-v',<br>
              Key = 'aqi/gfs/{}'.format('gfs.0p25.2019100612.f000.grib2'))['Body'].read()<br>
) if 'metre temp' in e[0]]

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

grid.wkt

grid.sort_values('location').wkt

p = pygrib.open(# 'gfs.0p25.2020072500.f000.grib2')#'gfs.0p25.2020010100.f000.grib2')<br>
    # 'gfs.0p25.2017072500.f000.grib2'<br>
    'gfs.0p25.2017072500.f000.grib2'<br>
)

In[ ]:

In[ ]:

cities = {<br>
 'Taipei': ( (121.5, 121.5), (25.0, 25) ),<br>
 'Delhi': ( (77.0, 77.25), (28.75, 28.5) ),<br>
 'LA': ((360-118.25, 360-117.75), (34.0, 34.0) )<br>
}

In[ ]:

In [ ]:
cities = {
    'LA': ((360 - 118.25, 360 - 117.75), (34.0, 34.0))
}

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic('time', '', 'coords = defaultdict(list)\n')

In[ ]:

for city in ['Delhi', 'Taipei', 'LA']:

In [ ]:
for city in ['LA']:
    gcity = [c for c in grid.location.unique(
    ) if 'Los A' in c][0] if city not in grid.location.unique() else city
    grid_points = grid[grid.location == gcity]
    for e in grid_points.itertuples():
        coords[city].append((e.grid_id, *np.array([(float(p.split(' ')[0]),
                                                    float(p.split(' ')[1]))
                                                   for p in e.wkt[10:].split(', ')[:4]]).mean(axis=0).tolist()))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

### Load Saved Features

In[ ]:

In [ ]:
def loadFiles(file):
    zd = zstd.ZstdDecompressor()
    data = []
    with tarfile.TarFile(file, 'r') as tar:
        assim_files = tar.getnames()
        for file in assim_files:
            if len(file) > 0:
                f = tar.extractfile(file)
                data.append(pickle.loads(zd.decompress(f.read())))
    return data

In[ ]:

In[ ]:

def listFiles(prefix, bucket = 'projects-v'):<br>
    paginator = s3.get_paginator('list_objects_v2')<br>
    page_iterator = paginator.paginate(Bucket = bucket,<br>
                                       Prefix = prefix )<br>
    files = []<br>
    for page in page_iterator:<br>
        files.extend([e['Key'] for e in page['Contents']])<br>
    return files

def listFiles(prefix):<br>
    return [os.path.join('cache/', prefix[4:], f)<br>
            for f in os.listdir('cache/' + prefix[4:])]

In[ ]:

In[ ]:

In[ ]:

def loadData(file):<br>
    zd = zstd.ZstdDecompressor()<br>
    data = pickle.loads(zd.decompress(open(file, 'rb').read()))<br>
    return data

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def flatten(l): return [e for s in l for e in s]

In[ ]:

loc_dict = {'la': 'Los Angeles (SoCAB)', 'tpe': 'Taipei', 'dl': 'Delhi'}

In [ ]:
loc_dict = {'la': 'Los Angeles (SoCAB)'}

In[ ]:

In [ ]:
rloc_dict = {v: k for k, v in loc_dict.items()}

In[ ]:

In [ ]:
tz_dict = grid.groupby('location').tz.first().to_dict()

In[ ]:

In [ ]:
loc_tz_dict = {rloc_dict[k]: v for k, v in tz_dict.items()}
loc_tz_dict

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

def downloadFile(file):<br>
    s3 = boto3.client('s3')<br>
    s3.download_file('projects-v', file, 'cache/' + file[4:])

def downloadFiles(files):<br>
    for path in set([f.split('/')[1] for f in files]):<br>
        os.makedirs('cache/{}'.format(path), exist_ok = True)

    Parallel(os.cpu_count() * 3)(delayed(downloadFile)(file)<br>
                                     for file in files)

In[ ]:

In[ ]:

downloadFiles(assim_files)

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time', '', "all_data_assim = loadFiles('cache/assim.tar')\n")

In[ ]:

In [ ]:
all_data_assim = flatten(all_data_assim)

In[ ]:

In [ ]:
len(all_data_assim)

In[ ]:

In[ ]:

In[ ]:

In [ ]:
rt_dict = {}

In[ ]:

In [ ]:
def process(d, e):
    t = d['time_end']
    if t not in rt_dict:
        rt = (t + 30).astype('datetime64[m]')
        rt_dict[t] = rt
    rt = rt_dict[t]
    grid_data_assim[grid_id][rt].update(e)
    grid_data_assim[grid_id][rt].update({'location': d['location']})

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    "grid_data_assim = defaultdict(lambda: defaultdict(dict))\nfor d in all_data_assim:\n    for grid_id, e in d['d1'].items():\n        process(d, e)\n")

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def processAssim(grid_id, v):
    df = pd.DataFrame.from_dict(v, orient='index')

    # tz
    assert df.location.nunique() == 1
    tz = loc_tz_dict[df.location.unique()[0]]
    df.index = df.index.tz_localize('UTC').tz_convert(tz).floor('1d')
    df.index.name = 'datetime'
    df.drop(columns='location', inplace=True)
    df = df.groupby(df.index).mean()  # .sort_index()#.head(10)

    # fill
    t = pd.to_datetime(submission.datetime.min()).tz_convert(tz).floor('1d')
    tf = pd.to_datetime(submission.datetime.max()).tz_convert(tz).floor('1d')
    df_t = set(df.index)
    extra_times = []
    while t <= tf:
        if t not in df_t:
            extra_times.append(t)
        t += datetime.timedelta(days=1)
    df_extra = pd.DataFrame(
        np.nan,
        index=pd.Series(extra_times),
        columns=df.columns)
    if not df_extra.empty:
        df = pd.concat([df, df_extra]).sort_index()
    else:
        # Optionally, handle the case where df_extra is empty
        # For example, you might simply want to ensure df is sorted by index
        df = df.sort_index()

    # ewm
    n = 1
    df_ewm1 = df.ewm(span=n).mean().astype(np.float32).fillna(df.mean())
    df_ewm1.columns = [c + '_{}day'.format(n) for c in df.columns]
    n = 2
    df_ewm2 = df.ewm(span=n).mean().astype(np.float32).fillna(df.mean())
    df_ewm2.columns = [c + '_{}day'.format(n) for c in df.columns]
    n = 5
    df_ewm5 = df.ewm(span=n).mean().astype(np.float32).fillna(df.mean())
    df_ewm5.columns = [c + '_{}day'.format(n) for c in df.columns]
    raw_df = df
    df = pd.concat((df_ewm1, df_ewm2, df_ewm5,
                    ), axis=1)

    # compile
    df.index = df.index.tz_localize(None)
    df.insert(0, 'grid_id', grid_id)
    return df

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    'all_dfs = Parallel(os.cpu_count())(delayed(processAssim)(grid_id, v)\n                                   for grid_id, v in grid_data_assim.items() )\n')

In[ ]:

In [ ]:
assim = pd.concat(all_dfs)
assim.index.name = 'datetime'
assim = assim.reset_index().set_index(['datetime', 'grid_id'])

In[ ]:

In [ ]:
del all_data_assim, grid_data_assim, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time', '', "all_data_tropomi = loadFiles('cache/tropomi-fine.tar')\n")

In[ ]:

In [ ]:
grid_data_tropomi = defaultdict(list)
for d in all_data_tropomi:
    for grid_id, e in d['d1'].items():
        # e = blend(e)#e.copy()
        e['datetime'] = d['time_end']
        e['location'] = d['location']
        grid_data_tropomi[grid_id].append(e)
    # break;

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic('time', '', "all_dfs = []\nfor grid_id, v in grid_data_tropomi.items():\n    df = pd.DataFrame(v)\n\n    # tz\n    assert df.location.nunique() == 1 \n    tz = loc_tz_dict[df.location.unique()[0]]\n    df.datetime = pd.to_datetime(df.datetime).dt.tz_convert(tz).dt.floor('1d')\n    df.drop(columns = 'location', inplace = True)\n\n    # group\n    for col in [c for c in df.columns if '_mean' in c]:\n        ct = np.where(df[col].isnull(), 0, df[col.replace('_mean', '_count')] )\n        df[col.replace('_mean', '_sum')] = df[col] * ct\n        df[col.replace('_mean', '_count')] = ct\n    df = df.groupby(df.datetime).sum()#.sort_index()#.head(10)\n    \n\n    for col in [c for c in df.columns if '_mean' in c]:\n        df[col] = (df[col.replace('_mean', '_sum')] \n                                / df[col.replace('_mean', '_count')]\n                    )\n    # filter\n    df = df[[c for c in df.columns if\n             ('.' in  c) and \n             ( ( '_mean' in c ) or ('column_stdev' in c) )\n            ]]\n    \n    # fill\n    t = pd.to_datetime(submission.datetime.min()).tz_convert(tz).floor('1d')\n    tf = pd.to_datetime(submission.datetime.max()).tz_convert(tz).floor('1d')\n    df_t = set(df.index)\n    extra_times = []\n    while t <= tf:\n        if t not in df_t:\n            extra_times.append(t)\n        t += datetime.timedelta(days = 1)\n    df_extra = pd.DataFrame(np.nan, index = pd.Series(extra_times), columns = df.columns )\n    if not df_extra.empty:\n        df = pd.concat([df, df_extra]).sort_index()\n    else:\n        # Optionally, handle the case where df_extra is empty\n        # For example, you might simply want to ensure df is sorted by index\n        df = df.sort_index()\n        \n    # clip\n    sigma = 4\n    high = df.ewm(span = 100).mean() + sigma * df.ewm(span = 100).std().fillna(100000)\n    df = df.clip(0, None)\n    df = np.minimum(df, high)\n    \n    # ewm     \n    n = 1\n    df_ewm1 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())\n    df_ewm1.columns = [c + '_{}day'.format(n) for c in df.columns]\n\n    n = 3\n    df_ewm3 = df.ewm(span = n).mean().astype(np.float32).fillna(df.mean())\n    df_ewm3.columns = [c + '_{}day'.format(n) for c in df.columns]\n    \n    raw_df = df\n    df = pd.concat((df_ewm1, df_ewm3, \n                   ), axis = 1)\n    \n    # compile\n    df.index = df.index.tz_localize(None)\n    df.insert(0, 'grid_id', grid_id)\n    all_dfs.append(df)\n")

In[ ]:

In [ ]:
tropomi = pd.concat(all_dfs)
tropomi.index.name = 'datetime'
tropomi = tropomi.reset_index().set_index(['datetime', 'grid_id'])

In[ ]:

In [ ]:
del all_data_tropomi, grid_data_tropomi, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic('time', '', '# downloadFiles(ifs_files)\n')

In[ ]:

In[ ]:

def loadData(f):<br>
    s3 = boto3.client('s3')<br>
    zd = zstd.ZstdDecompressor()<br>
    data = pickle.loads(zd.decompress(s3.get_object(Bucket = 'projects-v', Key = f)['Body'].read()))<br>
    return data

In[ ]:

In[ ]:

In[ ]:

ifs_files = sorted(listFiles('aqi/ifs/'));

ifs_tags = [<br>
    '128_015_aluvp', '128_134_sp', '128_136_tcw',<br>
    # '128_137_tcwv',<br>
'128_164_tcc',<br>
 '128_165_10u',<br>
'128_166_10v',<br>
 '128_167_2t',<br>
'128_168_2d',<br>
'128_206_tco3',<br>
'228_246_100u',<br>
'228_247_100v',<br>
]

ifs_files = [f for f in ifs_files # if any(z in f for z in ifs_tags)<br>
                    if '.oper.fc'  in f<br>
            ]

len(ifs_files)#[::1000]

In[ ]:

In[ ]:

In[ ]:

def loadData(f):<br>
    s3 = boto3.client('s3')<br>
    data = s3.get_object(Bucket = 'projects-v', Key = f)['Body'].read()<br>
    return data

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    '# all_data_ifs = Parallel(os.cpu_count() * 3)(delayed(loadData)(d) for d in \n#                         [f for f in ifs_files[::1] ])\n# # data = flatten(all_data)\n')

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic('time', '', "all_data_ifs = loadFiles('cache/ifs.tar')\n")

In[ ]:

In[ ]:

with tarfile.open('cache/ifs.tar', "w") as tar:<br>
    for file, data in zip(ifs_files[:], all_data_ifs[:]):<br>
        t = tarfile.TarInfo(name = file)<br>
        t.size = len(data)<br>
        b = io.BytesIO(data)<br>
        b.seek(0)<br>
        tar.addfile(<br>
            t, b)<br>
        # print(b)

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    "grid_data_ifs = defaultdict(lambda: defaultdict(dict))\nfor d in all_data_ifs:\n    for dt, v in d.items():\n        for grid_id, e in v.items():\n            # e['timezone'] = grid_tz_dict[grid_id]#d['location']\n            grid_data_ifs[grid_id][dt[0]].update(e)\n")

In[ ]:

In [ ]:
get_ipython().run_cell_magic('time', '', "all_dfs = []\n# def compileIfs(grid_id, v):\nfor grid_id, v in grid_data_ifs.items():\n    df = pd.DataFrame(v).T\n    df.index.name = 'datetime'\n    # df.sort_index(inplace = True)\n    df = df.reset_index()\n    df.sort_values('datetime', inplace = True)\n    \n    # break;\n    # extra_rows = []\n    # t = df.datetime.min()\n    # timestamps = set(df.datetime)\n    # t_max = df.datetime.max()\n    # while t < t_max:#max(timestamps):#df.datetime.max():\n    #     t_next = t + datetime.timedelta(seconds = 60 * 60 * 12) \n    #     if t_next not in timestamps:\n    #         if grid_id == '1X116': print(t_next)\n    #         extra = df[df.datetime == t].copy() #if not last else last.copy()\n    #         extra.datetime = t_next\n    #         extra_rows.append(extra)\n    #         # last = extra\n    #     # else:\n    #         # last = None\n    #     t = t_next;\n    # #if len(extra_rows) > 1 and grid_id == '1X116': \n    # #    print('{} extra rows'.format(len(extra_rows)))\n    # df = pd.concat((df, *extra_rows))\n    \n    df.set_index('datetime', inplace = True)    \n\n    # df = addExtraRows(df.reset_index()).set_index('datetime')    \n    df = df.ewm(span = 2 * 2).mean().astype(np.float32)\n    df.columns = [c + '_{}day'.format(2) for c in df.columns]\n\n        \n    # compile\n    df.index = df.index.tz_localize(None)\n    df.drop(columns = [c for c in df.columns if 'vapour' in c \n                       or 'mean0.05' in c \n                       or 'mean0.2' in c\n                       or 'mean1' in c\n                       or 'roughness' in c\n                       or 'albedo' in c\n                      ], inplace = True)\n    df.columns = ['ifs_'+ c for c in df.columns]\n    df.insert(0, 'grid_id', grid_id)\n    # return df;\n\n    all_dfs.append(df)\n    # break;\n    # break;\n")

all_dfs = Parallel(#os.cpu_count() * 2<br>
                  1)(delayed(compileIfs)(grid_id, v)<br>
                           for grid_id, v in grid_data_ifs.items())

In[ ]:

In [ ]:
ifs = pd.concat(all_dfs)
ifs.index.name = 'datetime'
ifs = ifs.reset_index().set_index(['datetime', 'grid_id'])

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
for i in [0, 10, -1]:
    df = all_dfs[i]
    df['grid_id'] = df['grid_id'].astype(str)
    grid['grid_id'] = grid['grid_id'].astype(str)  # Ensure grid_id is string
    try:
        location = grid.set_index('grid_id').location[df.grid_id.iloc[0]]
        print(location)
    except KeyError:
        print(f"Location for grid_id {df['grid_id'].iloc[0]} not found.")
        location = "Unknown Location"  # Default or placeholder location
    df_numeric = df.select_dtypes(include=[np.number])
    # plt.matshow(df_numeric.corr())
    # plt.colorbar()
    # plt.title(location)
    # plt.show()  # Ensure the plot is shown for each iteration
    # plt.matshow(df.corr()); plt.colorbar(); plt.title(str(location))

In[ ]:

In [ ]:
df.columns[1::6]

In[ ]:

In [ ]:
del all_data_ifs, grid_data_ifs, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

gfs_files = listFiles('aqi/gfs-5/')

len(gfs_files)

with tarfile.open('cache/gfs-5.tar', "w") as tar:<br>
    tar.add('cache/gfs-5/', arcname=os.path.sep)

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    "# all_data_gfs = Parallel(os.cpu_count() * 3)(delayed(loadData)(d) for d in \n#                         [f for f in gfs_files if 'f000' in f])\n# all_data_gfs = flatten(all_data_gfs)\n")

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    "all_data_gfs = loadFiles('cache/gfs-5.tar')\nall_data_gfs = flatten(all_data_gfs)\n")

In[ ]:

In [ ]:
len(all_data_gfs)

In[ ]:

In[ ]:

In[ ]:

In [ ]:
get_ipython().run_cell_magic(
    'time',
    '',
    "points = defaultdict(lambda: defaultdict(list))\nfor e in all_data_gfs:# [e for e in h if e[1] == city]:\n    t = datetime.datetime.strptime(e[0][-12:-2], '%Y%m%d%H')\n    n = ':'.join(e[0].split(':')[1:2])\n    city = e[1]\n    \n    arr = e[3] \n    arr2 = cv2.resize(arr, None, fx = 5, fy = 5,  )\n    \n    for grid_id, x, y in coords[city]:\n        v = arr2[ int(round(( e[2][0][1] - y ) / 0.05 + 2)), \n            int(round( ( (x % 360) - e[2][1][0] )/0.05 + 2 ) ) ]\n        points[grid_id][n].append((t, v, e[-1]))\n")

In[ ]:

In [ ]:
def addExtraRows(df):
    extra_rows = []
    t = df.datetime.min()
    timestamps = set(df.datetime)
    t_max = df.datetime.max()
    while t < t_max:  # max(timestamps):#df.datetime.max():
        t_next = t + datetime.timedelta(seconds=60 * 60 * 6)
        if t_next not in timestamps:
            extra = df[df.datetime == t].copy() if not last else last.copy()
            extra.datetime = t_next
            extra_rows.append(extra)
            last = extra
        else:
            last = None
        t = t_next
    if len(extra_rows) > 1:
        print('{} extra rows'.format(len(extra_rows)))
    return pd.concat((df, *extra_rows))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
all_dfs = []
for grid_point, grid_data in points.items():
    dfs = []
    for label, d in grid_data.items():
        if 'Volumetric soil moisture content' in label:
            continue
        df = pd.DataFrame(
            d,
            columns=[
                'datetime',
                label +
                '_local',
                label +
                '_city'])
        df.sort_values('datetime', inplace=True)
        dfs.append(df.set_index('datetime'))
    df = pd.concat(dfs, axis=1)
    df = addExtraRows(df.reset_index()).set_index('datetime')
    df = df.ewm(span=4 * 3).mean().astype(np.float32)
    df.columns = [c + '_{}day'.format(3) for c in df.columns]
    df['grid_id'] = grid_point
    all_dfs.append(df)
gfs = pd.concat(all_dfs)
gfs = gfs.reset_index().set_index(['datetime', 'grid_id'])
#
# print(df.corr().iloc[0, 1], label)
# break;

In[ ]:

In [ ]:
del all_data_gfs, points, all_dfs

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

### Compile

In[ ]:

In [ ]:
labels['dayofweek'] = labels.datetime.dt.dayofweek.astype(np.int32)

In[ ]:

In [ ]:
labels['dayinyear'] = labels.datetime.dt.dayofyear.astype(np.int32)

In[ ]:

In [ ]:
labels.head()

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def addGFS(labels):
    ldo = pd.to_datetime(labels.datetime).dt.floor('6h').dt.tz_localize(None)
    df = pd.concat((labels,
                    * [2 / 3 * gfs.reindex([ldo
                                            + datetime.timedelta(seconds=60 * 60 * 12),
                                            labels.grid_id]).reset_index(drop=True)
                        + 1 / 3 * gfs.reindex([ldo
                                               + datetime.timedelta(seconds=60 * 60 * 18),
                                               labels.grid_id]).reset_index(drop=True)
                       # for k, v in gfs_ewms.items()
                       ],
                    ), axis=1)
    return df

In[ ]:

In[ ]:

gfs.groupby(['datetime', 'grid_id']).nunique().mean(axis = 1).sort_values()[-50:]

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def ifsoffset12(c): return c.replace('ifs', 'ifs12')
def ifsoffset0(c): return c.replace('ifs', 'ifs0')

In[ ]:

In [ ]:
def addIFS(labels):
    ldo = pd.to_datetime(labels.datetime).dt.floor('12h').dt.tz_localize(None)
    df = pd.concat((labels,
                    * [ifs.reindex([ldo
                                    + datetime.timedelta(seconds=60 * 60 * 12),
                                    labels.grid_id]).reset_index(drop=True).rename(
                        columns=ifsoffset12),
                        ifs.reindex([ldo                            # + datetime.timedelta(seconds = 60 * 60 * 0)
                                     ,
                                     labels.grid_id]).reset_index(drop=True).rename(
                        columns=ifsoffset0)
                       # for k, v in gfs_ewms.items()
                       ],
                    ), axis=1)
    return df

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def addSat(labels, sat):
    tza = labels.datetime.copy()
    for location in labels.location.unique():
        t = pd.to_datetime(labels.datetime).dt.tz_convert(tz_dict[location])
        t = t.dt.floor('1d').dt.tz_localize(None)
        tza = np.where(labels.location == location, t, tza)
    tza = pd.to_datetime(pd.Series(tza, index=labels.index))
    df = pd.concat((labels,
                    sat.reindex([tza,
                                 labels.grid_id]).reset_index(drop=True)
                    ), axis=1)
    return df

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
all_data = addGFS(labels)
all_data = addIFS(all_data)
all_data = addSat(all_data, 'maiac' if dataset == 'pm' else tropomi)
if ASSIM:
    all_data = addSat(all_data, assim)
# all_data = addSat(all_data, misr)

In[ ]:

In [ ]:
all_data.tail()

In[ ]:

In [ ]:
all_data.head()

In[ ]:

In[ ]:

In[ ]:

In [ ]:
x = all_data[[c for c in all_data.columns if c not in [
    'datetime', 'value']]].copy()

In[ ]:

'7334C' - > '7F1D1'<br>
'HANW9' - > 'WZNCR'

In [ ]:
if dataset == 'tg':
    np.random.seed(SEED)
    x.loc[(x.grid_id == '7334C') & (
        np.random.random(len(x)) < 0.1), 'grid_id'] = '7F1D1'
    x.loc[(x.grid_id == 'HANW9') & (
        np.random.random(len(x)) < 0.1), 'grid_id'] = 'WZNCR'

In[ ]:

In [ ]:
x.grid_id = x.grid_id.astype('category')
x.location = x.location.astype('category')

In[ ]:

In [ ]:
y = all_data.value.astype(np.float32)
d = all_data.datetime

In[ ]:

In [ ]:
pickle.dump(all_data, open('cache/all_data_{}.pkl'.format(dataset), 'wb'))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
x.columns = [c.replace(':', '_') for c in x.columns]

In[ ]:

In [ ]:
x.shape

In[ ]:

In[ ]:

x.fillna(method = 'ffill')

x.isnull().sum()

x.isnull().sum().sort_values()

In[ ]:

In [ ]:
assert x.isnull().sum().sum() == 0

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
enet_params = {
    'alpha': [1e-2, 3e-2, 1e-1, 0.3, 1, 3, ],
    'l1_ratio': [0.01, 0.03, 0.1, 0.2, 0.5, 0.8, 0.9, ],
}

In[ ]:

In [ ]:
class PurgedKFold():
    def __init__(self, n_splits=5, gap=30):
        self.n_splits = n_splits
        self.gap = gap
    def get_n_splits(self, X, y=None, groups=None): return self.n_splits
    def split(self, X, y=None, groups=None):
        groups = groups.sort_values()
        X = X.reindex(groups.index)  # sort_values(groups)
        y = y.reindex(X.index)
        X, y, groups = sklearn.utils.indexable(X, y, groups)
        indices = np.arange(len(X))
        n_splits = self.n_splits
        for i in range(n_splits):
            test = indices[i * len(X) // n_splits: (i + 1)
                           * len(X) // n_splits]  # .index
            train = indices[(groups <= groups.iloc[test].min() -
                             datetime.timedelta(days=self.gap)) | (groups >= groups.iloc[test].max() +
                                                                   datetime.timedelta(days=self.gap))]  # .index
            yield train, test

In[ ]:

In [ ]:
class RepeatedPurgedKFold():
    def __init__(self, n_splits=5, n_repeats=1, gap=None):
        self.n_splits = n_splits
        self.n_repeats = n_repeats
        self.gap = gap
    def get_n_splits(self, X, y=None, groups=None):
        return self.n_splits * self.n_repeats + \
            self.n_repeats * (self.n_repeats - 1) // 2
    def split(self, X, y=None, groups=None):
        for i in range(self.n_repeats):
            for f in PurgedKFold(
                    self.n_splits + i,
                    gap=self.gap if self.gap else None).split(
                    X,
                    y,
                    groups):
                yield f

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def posRMSE(y, y_pred):
    return mean_squared_error(
        y,
        y_pred.clip(
            0,
            None) *
        2 /
        3 +
        y_pred *
        1 /
        3) ** 0.5

In [ ]:
pRMSE = make_scorer(posRMSE, greater_is_better=False)
SCORING = pRMSE

In[ ]:

In[ ]:

In[ ]:

In [ ]:
COLUMN_WIPE = 0  # len(x.columns) // 10

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def runENet(drop_cols=[], verbose=1):
    all_y_enet = []
    all_y_pred_enet = []
    enet_clfs = []
    enet_scalers = []
    for location in x.location.unique():
        if verbose > 0:
            print(location)
        x_loc = x[x.location == location].drop(columns=drop_cols)
        y_loc = y.reindex(x_loc.index)
        d_loc = d.reindex(x_loc.index)
        folds = list(
            PurgedKFold(
                4 if dataset == 'pm' else 3).split(
                x_loc,
                y_loc,
                d_loc))
        folds += [(np.arange(0, len(x_loc)), [])] * 2
        for train_fold, test_fold in folds:
            y_preds = []
            for i in range(8):
                scaler = StandardScaler()
                clf = ElasticNet(
                    max_iter=50000,
                    tol=1e-3,
                    selection='random',
                    precompute=True,
                    random_state=datetime.datetime.now().microsecond)
                model = RandomizedSearchCV(clf, enet_params,
                                           scoring=SCORING,
                                           # 'neg_root_mean_squared_error',
                                           cv=RepeatedPurgedKFold(random.randrange(3, 6),
                                                                  n_repeats=random.randrange(
                                                                      2, 4),
                                                                  gap=random.randrange(60, 120)),
                                           n_iter=random.randrange(4, 7),
                                           random_state=datetime.datetime.now().microsecond)
                # subset = train_fold#[:s].tolist() + train_fold[s +
                # l:].tolist()
                l = random.randrange(0, len(train_fold) // 10)
                s = random.randrange(0, len(train_fold) - l)
                subset = train_fold[:s].tolist() + train_fold[s + l:].tolist()
                xt = x_loc.iloc[subset, 2:].copy()
                # for c in misr.columns:
                if dataset == 'pm':
                    sample_cols = [
                        c for c in xt.columns if any(
                            z in c for z in [
                                'precision',
                                'air_mass',
                                'stdev'] + [
                                'no2',
                                'so2',
                                'co',
                                'o3',
                                'pm25_rh35_gcc',
                                'ifs'])]
                    xt[random.sample(sample_cols, k=random.randrange(
                        len(sample_cols) * 1 // 3, len(sample_cols) * 2 // 3))] = 0
                if dataset == 'tg':
                    if random.random() < 0.5:
                        xt[[c for c in xt.columns if 'precision' in c]] = 0
                    # if random.random() < 0.5: xt[[c for c in xt.columns if 'mean1' in c or 'mean2' in c]] = 0
                # xt[misr.columns] = 0
                # fj_drop =
                # if random.random() < {'Delhi': 0.5, 'Taipei': 0.2}.get(location, 0):
                #     xt[[c for c in xt.columns if 'FineMode' in c or 'Injection' in c]] = 0
                # # if random.random() < {'Delhi': 0.2, 'Taipei': 0.1}.get(location, 0):
                # #     xt[[c for c in xt.columns if 'Optical' in c]] = 0
                # if random.random() < {'Delhi': 0.85, 'Taipei': 0.5}.get(location, 0.2):
                #     xt[[c for c in xt.columns if c.startswith('ifs')]] = 0
                # xt = x_loc.iloc[subset].copy()
                xt[random.choices(xt.columns[2:], k=int(
                    round(random.random() * COLUMN_WIPE)))] = 0
                if random.random() < 0.3:
                    xt['dayinyear'] = 0
                model.fit(
                    pd.DataFrame(
                        scaler.fit_transform(xt).astype(
                            np.float32),
                        xt.index,
                        xt.columns),
                    y_loc.iloc[subset],
                    groups=d_loc.iloc[subset])
                enet_clfs.append(model.best_estimator_)
                enet_scalers.append(scaler)
                if i == 0 and verbose > 0:
                    display(
                        pd.DataFrame(
                            model.cv_results_).sort_values('rank_test_score').drop(
                            columns='params'))
                if len(test_fold) > 0:
                    y_pred = pd.Series(model.predict(
                        pd.DataFrame(scaler.transform(x_loc.iloc[test_fold, 2:]).astype(np.float32),
                                     columns=x_loc.columns[2:])),  # .clip(0, None),
                        index=y_loc.iloc[test_fold].index)
                    y_preds.append(y_pred)
            if len(test_fold) > 0:
                y_pred = pd.concat(y_preds)
                y_pred = y_pred.clip(
                    0,
                    None).groupby(
                    y_pred.index).mean().clip(
                    0,
                    None)
                y_pred = (y_pred.groupby(y_pred.index).mean())  # *2/3
                # + y_pred.groupby(y_pred.index).median() * 1/3 )
                all_y_enet.append(y_loc.iloc[test_fold])
                all_y_pred_enet.append(y_pred)
                if verbose >= 1:
                    print(location, round(np.corrcoef(
                        y_pred, all_y_enet[-1])[0, 1], 4))
                if verbose >= 3:
                    all_y_enet[-1].reset_index(
                        drop=True).ewm(span=10).mean().plot()
                    plt.plot(y_pred.reset_index(drop=True))
                    plt.title(location)
                    plt.figure()
                    print((x_loc.iloc[subset, 2:].std().clip(1, 1) *
                           model.best_estimator_.coef_).sort_values())
    return all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers

In[ ]:

In [ ]:
all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers = runENet(verbose=2)

In[ ]:

In[ ]:

In[ ]:

In [ ]:
os.makedirs('clfs_{}'.format(dataset), exist_ok=True)

In[ ]:

In [ ]:
pickle.dump((all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers),
            open('clfs_{}/enet_clfs_{}.pkl'.format(
                 dataset, run_label), 'wb'))

for i in range(3):<br>
    pd.Series(np.mean([e.coef_ for e in<br>
                   enet_clfs[len(enet_clfs) * i//3:len(enet_clfs) * (i + 1)//3]],<br>
        axis = 0), index = x.columns[2:]).sort_values().plot(kind = 'barh',<br>
                                                                figsize = (10, x.shape[1]// 4),<br>
                                                                title = list(x.location.unique())[i], )<br>
    plt.figure()

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

print(round(np.corrcoef( pd.concat(all_y_pred_enet).clip(0, None),#.reindex(df.index),<br>
                            pd.concat(all_y_enet)#.reindex(df.index)<br>
           )[0, 1], 3))

In[ ]:

In [ ]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef(pd.concat(all_y_pred_enet).clip(0, None).reindex(df.index),
                    pd.concat(all_y_enet).reindex(df.index))[0, 1]
    cs.append(c)
    print(location, round(c, 3))
print('\nBlend:', round(np.mean(cs), 3))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
lgb_params = {
    # [ 150, 200, 300, ],
    'n_estimators': np.arange(200, 400, 10) if dataset == 'pm' else np.arange(300, 600, 20),
    # [0.03, 0.05, 0.07, ],
    'learning_rate': np.arange(0.01, 0.04, 0.003) if dataset == 'pm' else np.arange(0.01, 0.061, 0.005),
    # [5, 7, 10, 15, 20,],
    'num_leaves': np.arange(4, 30) if dataset == 'pm' else np.arange(10, 30),
    'min_child_weight': np.arange(0.02, 0.1, 0.01),  # [  0.1, 0.2, ],
    'min_child_samples': [140, 170, 200, 250, 300, 400, 500, 600, 700, 850, 1000, 1400, ]
    if dataset == 'pm' else
                         [30, 40, 50, 60, 80, 100, 120, 150,
                             170, 200, 300, 500, 700, ],
    'reg_lambda': [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1,],
    'reg_alpha': [0, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, ],
    'linear_tree': [True, ],
    'subsample': np.arange(0.4, 0.901, 0.05),  # [0.3, 0.5,  0.8],
    'subsample_freq': [1],
    # [0.5, 0.8, ],#0.3, 0.5, 0.8],
    'colsample_bytree': np.arange(0.3, 0.71, 0.05) if dataset == 'pm' else np.arange(0.2, 0.81, 0.05),
    # [0.5, 0.8, ],#0.3, 0.5, 0.8],
    'colsample_bynode': np.arange(0.4, 1.01, 0.05) if dataset == 'pm' else np.arange(0.2, 1.01, 0.05),
    'linear_lambda': [1e-3, 3e-3, 1e-2, 3e-2, 0.1,],
    # 'max_bins': np.arange(120, 400, 20),
    # 'min_data_in_bin': np.exp(np.arange(np.log(3), np.log(12), 0.1)).astype(int), #np.arange(2, 10),# [2, 3, 4, 5, 10],
    # [ 192,
    # 255, 255, 384, 512],
    'min_data_per_group': [10, 20, 50, 100],
    'max_cat_threshold': [8, 16, 32, ],
    'cat_l2': [0.1, 1, 10],  # if dataset == 'pm' else [1e-3, 1e-2, 1e-1],
    'cat_smooth': [0.1, 1, 10],  # if dataset == 'pm' else [1e-3, 1e-2, 1e-1],
}

In[ ]:

In [ ]:
FAST = False
GAUSSIAN = 0.05  # if dataset == 'pm' else 0.1
COLUMN_WIPE = len(x.columns) // 5  # if dataset == 'pm' else 10)

In[ ]:

In[ ]:

In[ ]:

In [ ]:
def runLGB(drop_cols=[], verbose=1, n_bags=8, ):
    all_y_lgb = []
    all_y_pred_lgb = []
    tidx = 0
    lgb_clfs = []
    lgb_scalers = []
    for location in x.location.unique():
        random.seed(datetime.datetime.now().microsecond)
        np.random.seed(datetime.datetime.now().microsecond)
        if FAST and location != 'Delhi':
            continue
        if verbose > 0:
            print(location)
        x_loc = x[x.location == location].drop(
            columns=drop_cols)  # .iloc[:, :4]
        y_loc = y.reindex(x_loc.index)
        d_loc = d.reindex(x_loc.index)
        folds = list(
            (PurgedKFold(4) if dataset == 'pm' else PurgedKFold(
                3, gap=20)) .split(
                x_loc, y_loc, d_loc))
        folds += [(np.arange(0, len(x_loc)), [])] * 3
        for train_fold, test_fold in folds:
            y_preds = []
            for i in range(n_bags):  # if location == 'Delhi' else 4):
                model = RandomizedSearchCV(lgb.LGBMRegressor(seed=datetime.datetime.now().microsecond,
                                                             # n_jobs = 2, #
                                                             # os.cpu_count()
                                                             ), lgb_params,
                                           cv=RepeatedPurgedKFold(random.randrange(3, 6),
                                                                  n_repeats=random.randrange(
                                                                      1, 3),
                                                                  gap=random.randrange(60, 120)),
                                           n_iter=random.randrange(3, 5),
                                           scoring=SCORING,
                                           n_jobs=-1,  # os.cpu_count(),
                                           random_state=datetime.datetime.now().microsecond)
                l = random.randrange(0, len(train_fold) // 10)
                s = random.randrange(0, len(train_fold) - l)
                subset = train_fold[:s].tolist() + train_fold[s + l:].tolist()
                xt = x_loc.iloc[subset].copy()
                xt.iloc[:, 2:] += (GAUSSIAN * random.random() *
                                   np.random.default_rng().standard_normal(size=xt.iloc[:, 2:].shape)
                                   * xt.iloc[:, 2:].std().values[None, :])
                for c in random.choices(xt.columns[2:], k=int(
                        round(random.random() * COLUMN_WIPE))):
                    xt[c] = 0
                if dataset == 'pm' and random.random() < 0.3:
                    xt['dayinyear'] = 0
                sample_cols = [
                    c for c in xt.columns if any(
                        z in c for z in [
                            'precision',
                            'air_mass',
                            'stdev'] + [
                            'no2',
                            'so2',
                            'co',
                            'o3',
                            'pm25_rh35_gcc',
                            'ifs'])]
                xt[random.sample(sample_cols, k=random.randrange(
                    len(sample_cols) * 1 // 3, len(sample_cols) * 2 // 3))] = 0
                # if random.random() < {'Delhi': 0.7, 'Taipei': 0.2}.get(location, 0):
                #     xt[[c for c in xt.columns if 'FineMode' in c or 'Injection' in c]] = 0
                # if random.random() < {'Delhi': 0.1, 'Taipei': 0.1}.get(location, 0):
                #     xt[[c for c in xt.columns if 'Optical' in c]] = 0
                # if random.random() < {'Delhi': 1.0, 'Taipei': 0.2}.get(location, 0.2):
                #     xt[[c for c in xt.columns if c.startswith('ifs')]] = 0
                # if random.random() < {'Delhi': 0.9,}.get(location, 0.12):
                #     xt[[c for c in xt.columns if any(z + '_' in c for z in
                #          ['no2', 'so2', 'co', 'o3', 'pm25_rh35_gcc',  ] ) ]] = 0
                scaler = StandardScaler()
                xt.iloc[:, 2:] = scaler.fit_transform(
                    xt.iloc[:, 2:]).astype(np.float32)
                # for i in xt, y_loc.iloc[subset], d_loc.iloc[subset]:
                #     if i
                y_loc_subset = y_loc.iloc[subset].astype(np.int64)
                d_loc_subset = d_loc.iloc[subset].astype(np.int64) # //1e9
                print(f'xt = {xt}')
                print(f'y_loc.iloc[subset] = {y_loc_subset}')
                print(f'd_loc.iloc[subset] = {d_loc_subset}')
                model.fit(xt, y_loc_subset, groups=d_loc_subset,
                         )
                lgb_clfs.append(model.best_estimator_)
                lgb_scalers.append(scaler)
                if len(test_fold) > 0:
                    xtst = x_loc.iloc[test_fold].copy()
                    xtst.iloc[:, 2:] = scaler.transform(
                        xtst.iloc[:, 2:]).astype(np.float32)
                    y_pred = pd.Series(
                        model.predict(xtst),
                        index=y_loc.iloc[test_fold].index)
                    y_preds.append(y_pred)
                df = pd.DataFrame(model.cv_results_).sort_values(
                    'rank_test_score').drop(columns='params')
                if i == 0 and verbose > 1:  # df.mean_test_score.min() < -0:
                    display(df)
                    print()
            if len(test_fold) > 0:
                y_pred = pd.concat(y_preds)
                y_pred = (y_pred.groupby(y_pred.index).mean())  # * 2/3
                # + y_pred.groupby(y_pred.index).median() * 1/3 )
                all_y_lgb.append(y_loc.iloc[test_fold])
                all_y_pred_lgb.append(y_pred)
                if verbose >= 3:
                    all_y_lgb[-1].reset_index(
                        drop=True).ewm(span=10).mean().plot()
                    plt.plot(y_pred.reset_index(drop=True), linewidth=0.8)
                    try:
                        plt.plot(
                            all_y_pred_enet[tidx].clip(
                                0, None).reset_index(
                                drop=True))
                    except BaseException:
                        pass
                    plt.title(location)
                    plt.figure()
                tidx += 1
                if verbose > 0:
                    print(location, round(np.corrcoef(
                        y_pred, all_y_lgb[-1])[0, 1], 4))
    return all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers

In[ ]:

In [ ]:
all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers = runLGB(verbose=2, )

In[ ]:

In [ ]:
pickle.dump((all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers),
            open('clfs_{}/lgb_clfs_{}.pkl'.format(
                 dataset, run_label), 'wb'))

In[ ]:

In[ ]:

# print(round(np.corrcoef( pd.concat(all_y_pred_enet).clip(0, None), pd.concat(all_y_lgb))[0, 1], 3)  )<br>
print(round(np.corrcoef( pd.concat(all_y_pred_lgb).clip(0, None), pd.concat(all_y_lgb))[0, 1], 3) )<br>
print(round(np.corrcoef( pd.concat(all_y_pred_lgb).clip(0, None)<br>
                  +  0.1  * pd.concat(all_y_pred_enet).clip(0, None)<br>
                  , pd.concat(all_y_lgb))[0, 1], 4) )<br>
 # without weather ~0.79 lgb

In[ ]:

In [ ]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef(
        pd.concat(all_y_pred_lgb).clip(
            0,
            None).reindex(
            df.index) +
        0.1 *
        pd.concat(all_y_pred_enet).clip(
            0,
            None).reindex(
            df.index),
        pd.concat(all_y_lgb).reindex(
            df.index))[
        0,
        1]
    cs.append(c)
    print(location,
          round(c, 3))
print('\nBlend: ', round(np.mean(cs), 3))

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
len(lgb_clfs)

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In[ ]:

In [ ]:
submission = pd.read_csv('data_{}/submission_format.csv'.format(dataset))

In[ ]:

In [ ]:
submission['dayinyear'] = pd.to_datetime(submission.datetime).dt.dayofyear
submission['dayofweek'] = pd.to_datetime(submission.datetime).dt.dayofweek

In[ ]:

In [ ]:
submission['location'] = grid.set_index(
    'grid_id').location.reindex(submission.grid_id).values
submission['location'] = submission.location.astype('category')
submission['grid_id'] = submission.grid_id.astype(
    x.grid_id.dtype)  # 'category')

In[ ]:

In [ ]:
submission = addGFS(submission)
submission = addIFS(submission)
submission = addSat(submission, 'maiac' if dataset == 'pm' else tropomi)
if ASSIM:
    submission = addSat(submission, assim)

In[ ]:

In[ ]:

In[ ]:

In [ ]:
pickle.dump(submission, open('cache/submission_{}.pkl'.format(dataset), 'wb'))

In[ ]:

In[ ]:

In[ ]:

In [ ]:
xs = submission[x.columns]

In[ ]:

In [ ]:
submission.location.value_counts()

In[ ]:

In[ ]:

In[ ]:

In [ ]:
clf_path = 'clfs_{}'.format(dataset)
all_clfs = os.listdir(clf_path)

enet_tuples = [pickle.load(open( os.path.join(clf_path, f), 'rb'))<br>
                   for f in all_clfs if 'enet_clfs' in f] if dataset == 'pm' else []<br>
lgb_tuples = [pickle.load(open( os.path.join(clf_path, f), 'rb'))<br>
                   for f in all_clfs if 'lgb_clfs' in f]

In[ ]:

In [ ]:
enet_tuples = [(all_y_enet, all_y_pred_enet, enet_clfs, enet_scalers)]
lgb_tuples = [(all_y_lgb, all_y_pred_lgb, lgb_clfs, lgb_scalers)]

In[ ]:

In [ ]:
len(lgb_tuples)

In[ ]:

In [ ]:
len(enet_tuples)

labels

In[ ]:

In [ ]:
y_true = pd.concat(flatten([e[0] for e in lgb_tuples]))
y_true = y_true.groupby(y_true.index).mean()

In[ ]:

In [ ]:
y_pred_lgb = pd.concat(flatten([e[1] for e in lgb_tuples]))
y_pred_lgb = y_pred_lgb.groupby(y_pred_lgb.index).mean()  # * 2/3
# + y_pred_lgb.groupby(y_pred_lgb.index).median() * 1/3)

In[ ]:

In [ ]:
if dataset == 'pm':
    y_pred_enet = pd.concat(flatten([e[1] for e in enet_tuples]))
    y_pred_enet = (y_pred_enet.groupby(y_pred_enet.index).mean())
    # + y_pred_enet.groupby(y_pred_enet.index).mean() * 1/3)

In[ ]:

In [ ]:
cs = []
for location, df in x.groupby('location'):
    c = np.corrcoef(
        y_pred_lgb.clip(
            0, None).reindex(
            df.index) + 1 / 5 * (
                y_pred_enet.clip(
                    0, None).reindex(
                        df.index) if dataset == 'pm' else 0), y_true.reindex(
                            df.index))[
                                0, 1]
    cs.append(c)
    print(location,
          round(c, 3))
print('\nBlend: ', round(np.mean(cs), 3))

In[ ]:

In[ ]:

In[ ]:

[c.n_features_in_ for c in enet_scalers]

In[ ]:

In [ ]:
lgb_clfs, lgb_scalers, enet_clfs, enet_scalers = [], [], [], []
for i in range(3):
    lgb_clfs.extend(flatten([e[2][i * len(e[2]) // 3: (i + 1) * len(e[2]) // 3]
                             for e in lgb_tuples]))
    lgb_scalers.extend(flatten(
        [e[3][i * len(e[3]) // 3: (i + 1) * len(e[3]) // 3] for e in lgb_tuples]))
    enet_clfs.extend(flatten([e[2][i * len(e[2]) // 3: (i + 1) * len(e[2]) // 3]
                              for e in enet_tuples]))
    enet_scalers.extend(flatten(
        [e[3][i * len(e[3]) // 3: (i + 1) * len(e[3]) // 3] for e in enet_tuples]))

In[ ]:

In[ ]:

In[ ]:

In [ ]:
len(lgb_clfs)

In[ ]:

In [ ]:
lgb_ys = []
for clf_idx, clf in enumerate(lgb_clfs):
    x_loc = xs[xs.location == x.location.unique()[0]].copy()  # Always use the first (and only) location
    x_loc.iloc[:, 2:] = lgb_scalers[clf_idx].transform(x_loc.iloc[:, 2:])
    lgb_ys.append(pd.Series(clf.predict(x_loc), index=x_loc.index))
    lgb_y_combined = pd.concat(lgb_ys, axis=0)

In [ ]:
enet_ys = []
for clf_idx, clf in enumerate(enet_clfs):
    x_loc = xs[xs.location == x.location.unique()[0]].copy().iloc[:,2:]
    enet_ys.append(pd.Series(clf.predict(
        pd.DataFrame(enet_scalers[clf_idx].transform(x_loc), 
                     columns = x_loc.columns)), index = x_loc.index))
enet_ys = pd.concat(enet_ys)

lgb_ys.groupby(lgb_ys.index).std().plot()

In [ ]:
lgb_y = lgb_y_combined.groupby(lgb_y_combined.index).mean().clip(0, None)  # .sort_values()

In [ ]:
enet_y = enet_ys.groupby(enet_ys.index).mean().clip(0, None)  # .sort_values()

In [4]:
# if dataset == 'pm':<br>
# plt.scatter(enet_y, lgb_y, s= 0.1)<br>
print(round(np.corrcoef(enet_y, lgb_y)[0,1], 4) )#, s= 0.1)

NameError: name 'np' is not defined

In [ ]:
f = 8 if dataset == 'pm' else 100000  # lgb_y
ys = (lgb_y * (f - 1) / f + enet_y * 1 / f)

In [ ]:
ys.name = 'value'

In [ ]:
out = pd.concat((submission[['datetime', 'grid_id']],
                ys.reindex(submission.index)), axis=1)
out

In [ ]:
# out[pd.to_datetime(out.datetime) < datetime.datetime(2018, 1, 1)]

In [ ]:
# if dataset == 'pm' and ASSIM:<br>
#     pivot = '2018-01-10'<br>
#     b1 = pd.read_csv('submissions_pm/blend1.csv')<br>
#     mix = pd.concat((<br>
#         b1[b1.datetime < pivot],<br>
#         out[out.datetime >= pivot] ) )<br>
#     assert out.shape == mix.shape<br>
#     out = mix

In [ ]:
# for g in out.grid_id.unique():<br>
#     out[out.grid_id == g].set_index(pd.to_datetime(out[out.grid_id == g].datetime)).value.plot(<br>
#         marker = '.', linewidth = 0.1,)# markersize = 3)<br>
# plt.xlim(pd.to_datetime(submission.datetime).min(),<br>
# pd.to_datetime(submission.datetime).min() + datetime.timedelta(days =<br>
# 400));

In [ ]:
# for g in out.grid_id.unique():<br>
#     out[out.grid_id == g].set_index(pd.to_datetime(out[out.grid_id == g].datetime)).value.plot(<br>
#         marker = '.', linewidth = 0.1,)# markersize = 3)<br>
# plt.xlim(datetime.datetime(2020, 10, 15), datetime.datetime(2021, 9, 1));

In [ ]:
# if dataset == 'pm':<br>
#     sp = pd.read_csv('submissions_pm/lgb_baseline.csv')<br>
#     sp2 = pd.read_csv('submissions_pm/lgb_2.csv')<br>
#     g1 = pd.read_csv('submissions_pm/first_gfs.csv')<br>
#     m1 = pd.read_csv('submissions_pm/maiac.csv')<br>
#     m2 = pd.read_csv('submissions_pm/maiac2.csv')<br>
#     m3 = pd.read_csv('submissions_pm/maiac3.csv')<br>
#     b1 = pd.read_csv('submissions_pm/blend1.csv')<br>
#     b1f = pd.read_csv('submissions_pm/blend1f.csv')<br>
# else:<br>
#     sp = pd.read_csv('../submissions_tg/third_gfs.csv')<br>
#     s1 = pd.read_csv('../submissions_tg/sat1.csv')<br>
#     s2 = pd.read_csv('../submissions_tg/sat2.csv')<br>
#     s4 = pd.read_csv('../submissions_tg/sat4.csv')<br>
#     a1 = pd.read_csv('../submissions_tg/assim1.csv')<br>
#     n = pd.read_csv('../submissions_tg/new.csv')<br>
#     st1 = pd.read_csv('../submissions_tg/stack1.csv')<br>
#     n2 = pd.read_csv('submissions_tg/new.csv')

In [ ]:
# if dataset == 'pm':<br>
#     # plt.scatter(sp.value, out.value, s= 0.1);<br>
#     # plt.scatter(sp2.value, out.value, s= 0.1);<br>
#     # plt.scatter(g1.value, out.value, s= 0.1);<br>
#     plt.scatter(m1.value, out.value, s= 0.1);<br>
#     plt.scatter(m2.value, out.value, s= 0.1);<br>
#     plt.scatter(m3.value, out.value, s= 0.1);<br>
#     plt.scatter(b1.value, out.value, s= 0.1);<br>
#     plt.scatter(b1f.value, out.value, s= 0.1);<br>
# else:<br>
#     plt.scatter(sp.value, out.value, s= 0.1);<br>
#     plt.scatter(s1.value, out.value, s= 0.1);<br>
#     plt.scatter(s2.value, out.value, s= 0.1);<br>
#     plt.scatter(s4.value, out.value, s= 0.1);<br>
#     plt.scatter(a1.value, out.value, s= 0.1);<br>
#     plt.scatter(n.value, out.value, s= 0.1);<br>
#     plt.scatter(n2.value, out.value, s= 0.1)<br>
#     plt.scatter(st1.value, out.value, s= 0.1)

In [ ]:
# if dataset == 'pm':<br>
#     print(np.corrcoef((g1.value, m1.value, m2.value, m3.value, b1.value, b1f.value, out.value)).round(4))<br>
# else:<br>
#     print(np.corrcoef((sp.value, s1.value, s2.value, s4.value, a1.value, n.value, n2.value, st1.value, out.value)).round(4))

In [ ]:
out[out.datetime < '2021-04-01']

In [ ]:
os.makedirs('submissions_{}'.format(dataset), exist_ok=True)
out.to_csv('submissions_{}/new.csv'.format(dataset), index=False)